# Blockchain API: Get data for a single address

In [1]:
import pandas as pd
import altair as alt

In [2]:
pd.options.display.max_columns = 200
pd.options.display.max_rows = 1000
pd.set_option("display.max_colwidth", None)

### Call url

In [3]:
url = 'https://blockchain.info/rawaddr/1Bm9Eqyd46qhJH8pQi7W8QxDA5VhbtKbXK?format=json&limit=1000'

### Read the results

In [4]:
src = pd.read_json(url)

### How many records? 

In [5]:
len(src)

300

### The transaction specific data are in a nexted column called 'txs'

In [6]:
txs = pd.json_normalize(src['txs'])

### Deal with dates, times

In [7]:
txs['datetime'] = pd.to_datetime(txs['time'], unit='s')

In [8]:
txs['date'] = txs['datetime'].dt.date
txs['time'] = txs['datetime'].dt.time

### Only include fields that seem descriptive

In [9]:
txs_slim = txs[["datetime", 'date', 'time', 'result', 'balance', 'fee', 'hash', 'ver', 'vin_sz', 'vout_sz', 'size', 'weight', 'tx_index',
       'block_index', 'block_height', ]].copy()

In [10]:
txs_slim.head()

,datetime,date,time,result,balance,fee,hash,ver,vin_sz,vout_sz,size,weight,tx_index,block_index,block_height
0,2021-12-07 21:21:37,2021-12-07,21:21:37,1055733,20209519,541,fb4f06c6e58af5cc365a0109480e501eec172958828709e78861f3a8c013fd2d,2,2,1,344,722,1618079403810412,713104,713104
1,2021-12-07 16:30:10,2021-12-07,16:30:10,1666388,19153786,11245,46bedd9794ccf60a7cfed37e7f7d6d13ed3d508449c2159283a8c9d2278e61b1,1,1,79,2743,10645,6241041757395253,713064,713064
2,2021-12-07 09:31:01,2021-12-07,09:31:01,1921254,17487398,745,6c742784cfae27b6cd56c30fe0237302d1365d0dfe491a1675a3285dd2a19d13,1,3,1,491,992,690167862764820,713015,713015
3,2021-12-07 09:05:23,2021-12-07,09:05:23,2035140,15566144,338,63c485763d5de79bd4c32354527de408cc3d2fa86502da1e8c8b68922d5d2715,2,1,1,194,449,744281957616913,713013,713013
4,2021-12-07 07:17:22,2021-12-07,07:17:22,97814,13531004,540,53cec7da7caa99d8ec4abf1bee3411fbfb18fff27b95fa8c85836e7bdaa25297,2,2,2,374,845,5324197610876368,713003,713003


---

### Aggregate

In [11]:
txs_slim_grp = txs_slim.groupby(['date']).agg({'hash': 'count', 'fee':['mean', 'sum']}).round(2).reset_index()

In [12]:
txs_slim_grp.columns = ['_'.join(col).strip() for col in txs_slim_grp.columns.values]

In [13]:
txs_slim_grp.rename(columns={'date_':'date', 'hash_count': 'transaction_count'}, inplace=True)

In [14]:
txs_slim_grp['date'] = pd.to_datetime(txs_slim_grp['date'])

---

### Average daily transaction count

In [15]:
alt.Chart(txs_slim_grp).mark_bar().encode(
    x='date:T',
    y='transaction_count'
)

alt.Chart(...)

### Average daily transaction fee (not sure what this fee means yet)

In [16]:
alt.Chart(txs_slim_grp).mark_bar().encode(
    x='date:T',
    y='fee_mean'
)

alt.Chart(...)

---

### Export

In [17]:
txs_slim.to_csv('data/raw/transactions.csv', index=False)